<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/stress160.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7 Words with stress (F23)

[Wordlist 160](https://raw.githubusercontent.com/MK316/Myapps/main/data/stress160.csv)

In [ ]:
%%capture
!pip install gradio pandas gtts requests

In [ ]:
#@markdown Gradio link
import gradio as gr
import pandas as pd
import requests
from gtts import gTTS
from io import BytesIO

# URL of the raw CSV file on GitHub
csv_url = 'https://raw.githubusercontent.com/MK316/Myapps/main/data/stress160.csv'

# Use requests to get the CSV file content from GitHub
response = requests.get(csv_url)
assert response.status_code == 200, 'Failed to download CSV file'

# Load the CSV content into a DataFrame
df = pd.read_csv(BytesIO(response.content))

def generate_audio_by_id(word_id):
    word = df.loc[df['ID'] == word_id, 'Words'].values[0]
    return text_to_speech(word)

def generate_audio_by_word(word):
    if word in df['Words'].values:
        audio_path = text_to_speech(word)
        return audio_path, ""  # Return the path and an empty status message
    else:
        return None, "Word not found in the list."  # Return None and a status message


import os
import tempfile

def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as f:
        tts.save(f.name)
        return f.name  # Return the path to the saved file


def search_and_generate_audio(search_by, query):
    if search_by == 'ID':
        try:
            query = int(query)  # Convert ID to integer
            audio_path = generate_audio_by_id(query)
            return audio_path, ""
        except ValueError:
            return None, "ID must be an integer."
    elif search_by == 'Words':
        audio_path, status = generate_audio_by_word(query)
        return audio_path, status
    else:
        return None, "Please select a valid search option."


# Define the Gradio interface
iface = gr.Interface(
    fn=search_and_generate_audio,
    inputs=[
        gr.Radio(['ID', 'Words'], label="Search by:"),
        gr.Textbox(label="Enter ID or Word:")
    ],
    outputs=[
        gr.Audio(label="Audio of the word"),
        gr.Textbox(label="Status")
    ],
    title="Word Audio Generator"
)

# Launch the interface
iface.launch(debug=True)
